In [55]:
import yfinance as yf
import pandas as pd
import datetime as dt
import simfin as sf
import tensorflow as tf
import numpy as np

In [56]:
# First load the Simfin file containing the data

In [95]:
class Company():
    def __init__(self, ticker, market='us'):
        # Initialize the Simfin dataset
        sf.set_data_dir('C:Users\\Andi\\Desktop\\Udemy\\Python-Notebook\\Finance\\tickers')
        sf.set_api_key(api_key='free')
        
        # Attributes
        self._ticker = ticker
        self._company = yf.Ticker(self._ticker)
        self._market = market
        #self._market_cap
        #self._beta
        #self._shares_outstanding
        #self._growth_perp
        
        # Revenue
        self._revenue = self.income_sheet()['Revenue']
        
        self._growth_rate = self._revenue.pct_change(1)
        
        # Net Icome
        self._net_income = self.income_sheet()['Net Income']
        
         # Cash flow from operating activities (cahflow)
        self._CF_OA = self.cashflow()['Net Cash from Operating Activities']
        
         # Capital expenditures or also called Change in Fixed Assets & Intangibles (cashflow)
        self._CAPEX = self.cashflow()['Change in Fixed Assets & Intangibles']
        
        # Free cash flow computed from CF_OA - CAPEX (it is already negative CAPEX so we use a positive sign)
        self._FCF = self._CF_OA + self._CAPEX
        
        # Get the historical daily values since day 0
        self._history = self._company.history(period="max")
        self._history.drop(columns=['Dividends', 'Stock Splits'], inplace=True)
        
        # Logaritmic return of the stcok since day 0: 
        self._log_return = np.log(self._history['Close']/self._history['Close'].shift(1))
        
        # Sharpe Ratio and Annual SR
        self._sr = self.expected_return()/self.volatility()
        self._asr = self._sr*(252**0.5)
        
        # Expected Anual Volatility of the stock since day start_date
    def volatility(self, weights = 1):
        if weights == 1:
            return self._log_return.std()*252
        
        # Expected anual return 
    def expected_return(self):
        return self._log_return.mean()*252
        
        
    def normed_return(self, open_buy, start_date =0):
        if start_date !=0 and open_buy !=0:
            return self._history.loc[start_date:]['Close']/open_buy
        
        if start_date !=0 and open_buy ==0:
            return self._history.loc[start_date:]['Close']/self._history.iloc[0]['Close']
        
        if start_date ==0 and open_buy ==0:
            return self._history.loc['Close']/self._history.iloc[0]['Close']
        
        return self._history['Close']/open_buy
        
      
    def position(self, start_date=0, open_buy=0, amount_invested=1, allocation=1):
        position = self.normed_return(open_buy, start_date)*amount_invested*allocation
        daily_return = position.pct_change(1)
        daily_return_percentage = round(daily_return*100,2)
        df = pd.concat([position, daily_return, daily_return_percentage], axis=1)
        df.columns = ['Position '+self._ticker, 'Daily Return', 'Daily Return %']
        return df
        
    def income_sheet(self, time='annual'):
        columns = ['Revenue', 'Cost of Revenue', 'Gross Profit', 'Operating Expenses', 'Income Tax (Expense) Benefit, Net', 
                   'Operating Income (Loss)','Net Income','Pretax Income (Loss)']
        income_sheet_df = sf.load_income(variant=time, market= self._market).loc[self._ticker][columns]
        income_sheet_df.fillna(0, inplace=True)
        
        return income_sheet_df
        
    def balance_sheet(self,time='annual'):
        balance_sheet_df = sf.load_balance(variant=time, market= self._market).loc[self._ticker, 'Cash, Cash Equivalents & Short Term Investments':]
        balance_sheet_df.fillna(0,inplace=True)
                                                                                    
    
    
    def cashflow(self,time='annual'):
        cash_flow_df = sf.load_cashflow(variant=time, market= self._market).loc[self._ticker, 'Net Income/Starting Line':]
        cash_flow_df.drop(['Change in Accounts Receivable','Change in Other', \
                           'Change in Accounts Payable', 'Change in Inventories'], axis = 1, inplace=True)
        cahs_flow_df = cash_flow_df.fillna(0, inplace=True)
        return cash_flow_df
        
    def info(self):
        return self._company.info
        
    def plot():
        
        pass
    
    def valuation_method(self, method ="DCF"):
        
        def DCF(self):
            # We use the Discounted Cash Flow valuation model i order to estimate the Intrinsic Value
            pass
        
        if method == 'DCF':
            pass
        
    
        def Earnings_valuation(self):
            print("Earnings")
        
    
        
      
    
    def portfolio(stocks, allocation, amount):
          pass
    
    def estimate(self, n_years, method='linear'):
        
        def linear():
            pass
        
        def DNN():
            pass

In [96]:
aapl = Company("AAPL")


Dataset "us-income-annual" on disk (7 days old).
- Loading from disk ... Done!
Dataset "us-income-annual" on disk (7 days old).
- Loading from disk ... Done!
Dataset "us-cashflow-annual" on disk (6 days old).
- Loading from disk ... Done!
Dataset "us-cashflow-annual" on disk (6 days old).
- Loading from disk ... Done!


In [97]:
start = '2020-11-17'

In [98]:
aapl.position(start, 200, 100)

,Position AAPL,Daily Return,Daily Return %
Date,,,
2020-11-17,59.695000,NaN,NaN
2020-11-18,59.014999,-0.011391,-1.14
2020-11-19,59.320000,0.005168,0.52
2020-11-20,58.669998,-0.010958,-1.10
2020-11-23,57.382500,-0.021945,-2.19


In [99]:
aapl.volatility()

7.338526011984954

In [100]:
aapl.expected_return()

0.17621028781728237

In [102]:
aapl._asr

0.38117349390402316

In [125]:
cf = aapl.cashflow()
cf

Dataset "us-cashflow-annual" on disk (1 days old).
- Loading from disk ... Done!


,Net Income/Starting Line,Depreciation & Amortization,Non-Cash Items,Change in Working Capital,Net Cash from Operating Activities,Change in Fixed Assets & Intangibles,Net Change in Long Term Investment,Net Cash from Acquisitions & Divestitures,Net Cash from Investing Activities,Dividends Paid,Cash from (Repayment of) Debt,Cash from (Repurchase of) Equity,Net Cash from Financing Activities,Net Change in Cash
Report Date,,,,,,,,,,,,,,
2000-09-30,7.860000e+08,8.400000e+07,-1.940000e+08,1.920000e+08,868000000,-1.420000e+08,3.215000e+09,0.000000e+00,-9.720000e+08,0.000000e+00,0.000000e+00,-3.100000e+07,-3.100000e+07,-135000000
2001-09-30,-2.500000e+07,1.000000e+08,-1.010000e+08,2.110000e+08,185000000,-2.320000e+08,5.151000e+09,-1.900000e+07,8.920000e+08,0.000000e+00,0.000000e+00,4.200000e+07,4.200000e+07,1119000000
2002-09-30,6.500000e+07,1.140000e+08,2.200000e+07,-1.120000e+08,89000000,-1.740000e+08,2.871000e+09,-5.200000e+07,-2.520000e+08,0.000000e+00,0.000000e+00,1.050000e+08,1.050000e+08,-58000000
2003-09-30,6.900000e+07,1.130000e+08,-1.100000e+07,1.190000e+08,290000000,-1.640000e+08,2.491000e+09,0.000000e+00,8.280000e+08,0.000000e+00,0.000000e+00,2.700000e+07,2.600000e+07,1144000000
2004-09-30,2.660000e+08,1.500000e+08,1.500000e+08,3.680000e+08,934000000,-1.760000e+08,1.141000e+09,0.000000e+00,-1.488000e+09,0.000000e+00,-3.000000e+08,4.270000e+08,1.270000e+08,-427000000
2005-09-30,1.328000e+09,1.790000e+08,5.360000e+08,4.920000e+08,2535000000,-2.600000e+08,8.609000e+09,0.000000e+00,-2.556000e+09,0.000000e+00,0.000000e+00,5.430000e+08,5.430000e+08,522000000
2006-09-30,1.989000e+09,2.250000e+08,2.310000e+08,-2.250000e+08,2220000000,-6.850000e+08,7.201000e+09,0.000000e+00,3.570000e+08,0.000000e+00,0.000000e+00,-3.700000e+07,3.240000e+08,2901000000
2007-09-30,3.496000e+09,3.170000e+08,3.290000e+08,1.325000e+09,5467000000,-9.860000e+08,-2.312000e+09,0.000000e+00,-3.249000e+09,0.000000e+00,0.000000e+00,3.650000e+08,7.420000e+08,2960000000
2008-09-30,6.119000e+09,4.960000e+08,9.360000e+08,2.045000e+09,9596000000,-1.199000e+09,-6.760000e+09,-2.200000e+08,-8.189000e+09,0.000000e+00,0.000000e+00,4.830000e+08,1.116000e+09,2523000000


In [129]:
ias = aapl.income_sheet()

Dataset "us-income-annual" on disk (1 days old).
- Loading from disk ... Done!


In [130]:
ias

,Revenue,Cost of Revenue,Gross Profit,Operating Expenses,"Income Tax (Expense) Benefit, Net",Operating Income (Loss),Net Income,Pretax Income (Loss)
Report Date,,,,,,,,
2000-09-30,7.983000e+09,-5.817000e+09,2.166000e+09,-1.546000e+09,-3.060000e+08,6.200000e+08,786000000,1092000000
2001-09-30,5.363000e+09,-4.128000e+09,1.235000e+09,-1.568000e+09,1.500000e+07,-3.330000e+08,-37000000,-52000000
2002-09-30,5.742000e+09,-4.139000e+09,1.603000e+09,-1.555000e+09,-2.200000e+07,4.800000e+07,65000000,87000000
2003-09-30,6.207000e+09,-4.499000e+09,1.708000e+09,-1.683000e+09,-2.400000e+07,2.500000e+07,68000000,92000000
2004-09-30,8.279000e+09,-6.022000e+09,2.257000e+09,-1.921000e+09,-1.040000e+08,3.360000e+08,266000000,370000000
2005-09-30,1.393100e+10,-9.889000e+09,4.042000e+09,-2.399000e+09,-4.800000e+08,1.643000e+09,1328000000,1808000000
2006-09-30,1.931500e+10,-1.371700e+10,5.598000e+09,-3.145000e+09,-8.290000e+08,2.453000e+09,1989000000,2818000000
2007-09-30,2.400600e+10,-1.585200e+10,8.154000e+09,-3.745000e+09,-1.512000e+09,4.409000e+09,3496000000,5008000000
2008-09-30,3.749100e+10,-2.429400e+10,1.319700e+10,-4.870000e+09,-2.828000e+09,8.327000e+09,6119000000,8947000000


In [136]:
aapl.valuation_method()

UnboundLocalError: local variable 'DCF' referenced before assignment